# Optimización parámetrica

**Problema a resolver**
- Se encontró el modelo de machine learning que parece funcionar.
- Pero…. Es necesario optimizar el modelo y encontrar los mejores parámetros, y así mejorar las métricas.


⚠️ Los modelos en Scikit-Learn tienen bastantes parámetros para configurar dichos modelos, por lo cuál es fácil perdernos y bastante difícil medir la sensibilidad de los mismos manualmente.


**Enfoques de Optimización Parámetrica**
1. **Optimización por matriz de parámetros** (`GridSearchCV`) <br>
📢 Utilizar cuando se quiere hacer una prueba a fondo, y se cuente con el tiempo y poder de procesamiento.
    1. Definir una o varias métricas a optimizar.
    2. Identificar posibles valores para los parámetros.
    3. Crear un diccionario de parámetros.
    4. Usar Cross Validation.
    5. Entrenar el modelo.

	> 💡 Opción sistemática, organizada y exhaustiva que nos dice cuál es la mejor combinación de parámetros y valores para entrenar al modelo.

2. **Optimización por búsqueda aleatoria** (`RandomizedSearchCV`) <br>
📢 Utilizar cuando se quieran explorar posibles optimizaciones, o cuando se tenga poco poder de procesamiento.
    1. Definir una o varias métricas a optimizar.
    2. Identificar el rango de valores que pueden tomar los parámetros.
    3. Crear diccionario con los rangos.
    4. Usar Cross Validation. 
    5. Entrenar el modelo.

	> 💡 Ideal para cuando no se tiene el tiempo para implementar `GridSearchCV`.

## Implementación

Librerias:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor # Meta estimador a optimizar
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # Opimizador por búsqueda aleatoria

In [2]:
%run 0.0-ml_professional-setup.ipynb

Carga del dataset `World Happiness` (2017).

In [3]:
file_path = path.data_raw_dir("2017.csv")
df = pd.read_csv(file_path)
df.columns = ["country","rank","score","high","low","gdp","family","lifexp","freedom","generosity","corruption","dystopia"]
df.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


Quitamos las variables categorícas y creamos un dataset con los features y otro con los targets.


📌 Eliminamos el feature `rank` del dataset ya que esta fuertemente correlacionada con el target `score` (describe los mismo), incluirla en el entrenamiento podría generar sesgos.

In [4]:
X = df.drop(columns=["country", "score", "rank"], axis=1)
y = df["score"]

### `RandomizedSearchCV`

Configuramos el optimizador:

Parámetros de `RandomForestRegressor` a optimizar:
- `n_estimators`: Número de árboles de decisión que van a componer el "bosque aleatorio".
- `criterion`: Medida de calidad de los splits hacen los árboles generados. Nos dice que tan bueno o malo es.
- `max_depth`: Profundidad de lo árboles.


Parámetros del optimizador:
-	`estimator`: Instancia del estimador/modelo que queremos optimizar.
- `param_distributions`: Diccionario de parámetros a optimizar.
- `n_iter`: Número máximo de muestreos que hace sobre los parámetros.
- `cv`: Cross Validation, número de folds para `KFold`.
- `scoring`: Medir la presición del modelo.

In [5]:
# Estimador
reg = RandomForestRegressor()

# Parámetros a optimizar a partir de un determinado rango
params = {
	"n_estimators": range(4, 16),
	"criterion": ["squared_error", "absolute_error"],
	"max_depth": range(2, 10)
}

# Generamos la optimización
rand_est = RandomizedSearchCV(estimator=reg, param_distributions=params, n_iter=10, cv=3, scoring="neg_mean_absolute_error").fit(X, y)

¿Cuál fue el mejor estimador? y ¿Cuáles fueron los mejores parámetros?

In [6]:
print("Mejor estimador:", rand_est.best_estimator_)
print("Mejores parámetros:", rand_est.best_params_)

# Probamos la predicciones
print("Predicción:", rand_est.predict(X.iloc[[0]]))
print("Valor real:", y[0])

Mejor estimador: RandomForestRegressor(criterion='absolute_error', max_depth=7, n_estimators=14)
Mejores parámetros: {'n_estimators': 14, 'max_depth': 7, 'criterion': 'absolute_error'}
Predicción: [7.50571428]
Valor real: 7.53700017929077


### `GridSearchCV`

Para el mismo meta estimador `Random Forest`, pero utilizando el optimizador Grid Search. Los parámetros a optimizar son los mismos que en Randomized Search, sin embargo en lugar de enviarle un rango de posibles valores, le mandamos un array con los valores a testear.

In [7]:
# Estimador
reg_gs = RandomForestRegressor()

# Parámetros a optimizar a partir de un determinado rango
params_gs = {
	"n_estimators": [4, 8, 15],
	"criterion": ["squared_error", "absolute_error"],
	"max_depth": [2, 4, 6, 8, 10]
}

gs_est = GridSearchCV(estimator=reg_gs, param_grid=params_gs, scoring="neg_mean_squared_error", cv=3).fit(X, y)

Verificamos cuál fue el mejor estimador y os mejores parámetros:

In [8]:
print("Mejor estimador:", gs_est.best_estimator_)
print("Mejores parámetros:", gs_est.best_params_)

# Probamos la predicciones
print("Predicción:", gs_est.predict(X.iloc[[0]]))
print("Valor real:", y[0])

Mejor estimador: RandomForestRegressor(criterion='absolute_error', max_depth=10, n_estimators=8)
Mejores parámetros: {'criterion': 'absolute_error', 'max_depth': 10, 'n_estimators': 8}
Predicción: [7.50750011]
Valor real: 7.53700017929077


Tanto `RandomizedSearchCV`  como `GridSearchCV` nos permiten encontrar la mejor configuración para un modelo de manera semi-automatizada.